In [ ]:
# run once
# ! pip install gspread oauth2client df2gspread boto3
from dcicutils import ff_utils
from functions.notebook_functions import *
import pandas as pd

my_key = get_key('koray_data')

all_users = ff_utils.search_metadata('/search/?type=User', key = my_key)
all_users_with_lab = [i for i in all_users if i.get('lab')]


all_labs = ff_utils.search_metadata('/search/?type=Lab', key = my_key)
#skip some labs
skip_title = ['Peter Park, HARVARD', 'DCIC Testing Lab']
all_labs = [i for i in all_labs if i['display_title'] not in skip_title]

all_grants = ff_utils.search_metadata('/search/?type=Award', key = my_key)
attribution = {'users':all_users,
               'labs': all_labs,
               'grants': all_grants}

In [ ]:
def extract_data(user, record):
    # Columns
    # First Name	Last Name	Account Email	Role 4DN	Role OH	Notes    Other Email	
    info_list = []
    info_list.extend([user['first_name'], user['last_name'], user['email']])
    if record:
        info_list.append(user.get('job_title', ""))
        info_list.append(record['Role'])
        info_list.append(record['Notes'])
        #get extra mail addresses
        extra_mail = record['Additional email']
        if user['email'] != record['Email']:
            if extra_mail:
                extra_mail = record['Email'] + ', ' + extra_mail
            else:
                extra_mail = record['Email'] 
        info_list.append(extra_mail)
        return info_list
    
    # if did not exist on OH list
    else:
        info_list.append(user.get('job_title', ""))
        info_list.append("")
        info_list.append("Added from 4DN Data Portal")
        info_list.append("")
        return info_list

def get_old_info(userlist_unsorted, excel_info):
    userlist = []
    userlist_ordered = sorted(userlist_unsorted, key=lambda k: k.get('job_title', 'z'))
    PI_list = [i for i in userlist_ordered if i.get('job_title', 'z').lower() == 'pi']
    coI_list = [i for i in userlist_ordered if i.get('job_title', 'z').lower() in ['co-I', 'co-pi']]
    other = [i for i in userlist_ordered if i.get('job_title', 'z').lower() not in ['pi', 'co-I', 'co-pi']]
    userlist = PI_list + coI_list + other
    
    print_data = []
    for a_user in userlist:
        res1 = [i for i in excel_info if i['First Name'] == a_user['first_name'] if i['Last Name'] == a_user['last_name']]
        if res1:
            print_data.append(extract_data(a_user, res1[0]))
            excel_info = [i for i in excel_info if not (i['Email'] == res1[0]['Email'])]
            continue
            
        res2 = [i for i in excel_info if i['Last Name'] == a_user['first_name'] if i['First Name'] == a_user['last_name']]
        if res2:
            print_data.append(extract_data(a_user, res2[0]))
            excel_info = [i for i in excel_info if not (i['Email'] == res2[0]['Email'])]
            continue
        
        res3 = [i for i in excel_info if i['Email'] == a_user['email']]
        if res3:
            print_data.append(extract_data(a_user, res3[0]))
            excel_info = [i for i in excel_info if not (i['Email'] == res3[0]['Email'])]
            continue
        
        res4 = [i for i in excel_info if i['Additional email'] == a_user['email']]
        if res4:
            print_data.append(extract_data(a_user, res4[0]))
            excel_info = [i for i in excel_info if not (i['Email'] == res4[0]['Email'])]
            continue
        
        else:
            print_data.append(extract_data(a_user, {}))
            
    return print_data, excel_info
            
            



In [ ]:
# GET OH SHEET
import boto3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import json

# GET KEY FROM S3 
s3 = boto3.resource('s3')
obj = s3.Object('elasticbeanstalk-fourfront-webprod-system', 'DCICjupgoogle.json')
cont = obj.get()['Body'].read().decode()
key_dict = json.loads(cont)
SCOPES = 'https://www.googleapis.com/auth/spreadsheets.readonly'
creds = ServiceAccountCredentials.from_json_keyfile_dict(key_dict, SCOPES)
gc = gspread.authorize(creds)

# The ID and range of a sample spreadsheet.
book_id = '117ZcpSrbi7sxCgUaSXnMWlt29K2bbS_jCr9XaOuFfmQ'
sheet_names = ['DCIC & OH', 'IT', 'NT', 'NOFIC', 'NBC','TCPA']
book = gc.open_by_key(book_id)


for name in sheet_names:
    print("-----------------------------")
    print(name)
    worksheet = book.worksheet(name)
    table = worksheet.get_all_values()
    ##Convert table data into a dataframe
    df = pd.DataFrame(table[1:], columns=table[0])
    row_list = df.to_dict(orient='records')
    
    # make small format adjustments
    for a_row in row_list:
        a_row.update((k, v.strip()) for k, v in a_row.items())
        up_dict = {'Email': a_row['Email'].lower(),
                   'Additional email': a_row['Additional email'].lower()}
        a_row.update(up_dict)

    #awards in scope
    sheet_awards = [i.replace(' ', '').strip() for i in df['Project Number'].unique() if i.strip()]
    sheet_awards = list(set(sheet_awards))
    for sheet_award in sheet_awards:
        my_award = [i for i in all_grants if i['name'] == sheet_award][0]
        #get_grant_pi his/her lab
        grant_pi = my_award['pi']['display_title']
        grant_pi_lab = my_award['pi']['lab']['display_title']
        center_name = '{:10} ({})'.format(my_award['center_title'], my_award['name'])
        #print(print_list[0])
        
        sheet_labs = []
        # find all labs with that award
        for a_lab in all_labs:
            if not a_lab.get('awards'):
                continue
            if my_award['uuid'] in [i['uuid'] for i in a_lab['awards']]:
                # bring the award lab to the top
                if a_lab['display_title'] == grant_pi_lab:
                    sheet_labs.insert(0, a_lab)
                else:
                    sheet_labs.append(a_lab)
        
        # for each lab, get all users from our database
        for my_lab in sheet_labs:
            #high level info - award and lab
            high_info = [center_name, my_lab['display_title']]
            #user info
            lab_users = [i for i in all_users_with_lab if i['lab']['uuid'] == my_lab['uuid']]
            print_data, row_list = get_old_info(lab_users, row_list)
            #combine
            print_data = [high_info + i for i in print_data]
            
            for i in print_data:
                print(';'.join(i)) 
        print('#######')
    

    # anything that is not used from the excel
    if row_list:
        print('##################')
        for a_row in row_list:
            if not a_row['Last Name']:
                continue
            print_line = [a_row['Project Number'], a_row['PI Contact'], a_row['First Name'], 
                          a_row['Last Name'], a_row['Email'],"", a_row['Role'], a_row['Notes'], 
                          a_row['Additional email']]
            print(';'.join(print_line)) 
            

In [ ]:
print(all_users[0])